In [ ]:
pip install opencv-python pillow easyocr ultralytics deepface tf-keras

In [3]:
import json
import os
import pandas as pd
import isodate
import requests
import numpy as np
import cv2
from collections import Counter
from sklearn.cluster import KMeans
import easyocr
import torch
import webcolors
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import random

C:\Users\duwjd\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [31]:
with open('video_info.json', "r", encoding="utf-8") as file:
        data = json.load(file)

flattened_data = [video for sublist in data for video in sublist]
df = pd.DataFrame(flattened_data)

def convert_duration(iso_duration):
    if pd.isna(iso_duration):  # NaN 값 처리
        return "00:00:00"
    if not isinstance(iso_duration, str):  # 문자열이 아닐 경우 처리
        return "00:00:00"

    try:
        duration = isodate.parse_duration(iso_duration)
        total_seconds = int(duration.total_seconds())
        return f"{total_seconds // 3600:02}:{(total_seconds % 3600) // 60:02}:{total_seconds % 60:02}"
    except Exception as e:
        print(f"Error parsing duration: {iso_duration} -> {e}")
        return "00:00:00"

# duration 변환 적용
df['duration'] = df['duration'].apply(convert_duration)

# 1분 이상인 데이터 필터링
df = df[df['duration'] > "00:01:00"]

In [ ]:
category_mapping = {
    '1': "영화/드라마",
    '2': "자동차",
    '10': "음악",
    '15': "동물",
    '17': "스포츠",
    '19': "여행",
    '20': "게임",
    '22': "일상/브이로그",
    '23': "엔터테인먼트",
    '24': "엔터테인먼트",
    '25': "뉴스/정치",
    '26': "스타일",
    '27': "교육",
    '28': "과학/기술",
}

df["category"] = df["category_id"].map(category_mapping)
df = df.drop(columns=["category_id"])
df.to_csv("video_info_2.csv", index=False, encoding="utf-8-sig")

In [47]:
df=pd.read_csv("video_datas_2.csv", encoding="utf-8-sig")
df_total=pd.read_csv('channel_info.csv',encoding='utf-8-sig')
df['channel_id'] = df['channel_id'].astype(str)
df_total['channel_id'] = df_total['channel_id'].astype(str)
df=pd.merge(left=df, right=df_total, how='inner', on='channel_id')
df

,title,video_id,published_date,thumbnail_url,view_count,like_count,comment_count,duration,channel_id,category,subscriber_count,total_video_count
0,8남매에게 집을 새집으로 바꿔줬습니다,1zpA-hwn18I,2024-07-07,https://i.ytimg.com/vi/1zpA-hwn18I/hqdefault.jpg,10820141.0,237481.0,15668.0,00:08:53,"['보겸TV', 'UCu9BCtGIEr73LXZsKmoujKw']",엔터테인먼트,1560만,9.3천
1,Stray Kids &quot;Chk Chk Boom&quot; M/V,0P0aQreFs8w,2024-07-19,https://i.ytimg.com/vi/0P0aQreFs8w/hqdefault.jpg,145455906.0,3186101.0,398776.0,00:03:26,"['JYP Entertainment', 'UCaO6TYtlC8U5ttz62hTrZgg']",음악,3040만,2.3천
2,Stray Kids &quot;Lose My Breath (Feat. Charlie...,SQ1yPMTIwCU,2024-05-10,https://i.ytimg.com/vi/SQ1yPMTIwCU/hqdefault.jpg,61506319.0,1535745.0,168777.0,00:02:53,"['JYP Entertainment', 'UCaO6TYtlC8U5ttz62hTrZgg']",음악,3040만,2.3천
3,NAYEON &quot;ABCD&quot; M/V,oUZttxRcPZw,2024-06-14,https://i.ytimg.com/vi/oUZttxRcPZw/hqdefault.jpg,59024605.0,1573717.0,171316.0,00:03:42,"['JYP Entertainment', 'UCaO6TYtlC8U5ttz62hTrZgg']",음악,3040만,2.3천
4,TWICE “Strategy (feat. Megan Thee Stallion)” M/V,Sz_wWzgh-vQ,2024-12-06,https://i.ytimg.com/vi/Sz_wWzgh-vQ/hqdefault.jpg,57327700.0,1392001.0,159633.0,00:03:51,"['JYP Entertainment', 'UCaO6TYtlC8U5ttz62hTrZgg']",음악,3040만,2.3천
...,...,...,...,...,...,...,...,...,...,...,...,...
618,[디지몬 슈퍼럼블] 대규모 업데이트 신규지역: 데이터 세계로 !,Z6RHGgr9I4E,2025-02-19,https://i.ytimg.com/vi/Z6RHGgr9I4E/hqdefault.jpg,270903.0,156.0,50.0,00:01:47,"['Move Interactive : 무브인터렉티브', 'UCIqdAriO3NR9V...",게임,1.06만,71
619,[디지몬 슈퍼럼블] 다가온 최.종.장 신규 지역 스파이럴 마운틴 업데이트!,8-HH6nqyuNs,2024-05-29,https://i.ytimg.com/vi/8-HH6nqyuNs/hqdefault.jpg,13968.0,101.0,35.0,00:01:33,"['Move Interactive : 무브인터렉티브', 'UCIqdAriO3NR9V...",게임,1.06만,71
620,[디지몬 슈퍼럼블] 궁극 진화! 황제드라몬 강림!,1G6ulF3Z2yA,2024-08-29,https://i.ytimg.com/vi/1G6ulF3Z2yA/hqdefault.jpg,10148.0,64.0,19.0,00:01:01,"['Move Interactive : 무브인터렉티브', 'UCIqdAriO3NR9V...",게임,1.06만,71
621,파피 플레이타임 요약 통합본,1bqvwBgDhkk,2024-03-25,https://i.ytimg.com/vi/1bqvwBgDhkk/hqdefault.jpg,34019799.0,322525.0,7726.0,00:08:50,"['소맥거핀', 'UCaaZFxjlHI3T_I-uFbldIGA']",영화/드라마,944만,468


In [197]:
url='https://i.ytimg.com/vi/FR4E9DPYG3A/hqdefault.jpg'
response = requests.get(url, stream=True, timeout=5)
response.raise_for_status()
image = np.asarray(bytearray(response.content), dtype=np.uint8)
image = cv2.imdecode(image, cv2.IMREAD_COLOR)
# 이미지 크기 얻기
height, width, _ = image.shape  # height와 width 값을 가져옵니다.
print(f"Image Width: {width}, Image Height: {height}")

Image Width: 480, Image Height: 360


## 채널 정보 크롤링

In [3]:
with open('channel_ids.json', "r", encoding="utf-8") as file:
        data = json.load(file)
len(data)

602

In [139]:
import ast
df=pd.read_csv("video_datas.csv", encoding="utf-8-sig")
data=list(df.iloc[:,8])
data = [ast.literal_eval(item)[1] for item in data]
data=data[3501:]
len(data)

6180

In [143]:
# 여러 개의 User-Agent 리스트
USER_AGENTS = [
    # Chrome (Windows)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",

    # Chrome (Mac)
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    
    # Chrome (Linux)
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0",
    
    # Firefox (Windows)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:115.0) Gecko/20100101 Firefox/115.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:102.0) Gecko/20100101 Firefox/102.0",
    
    # Firefox (Mac)
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11.6; rv:94.0) Gecko/20100101 Firefox/94.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; rv:102.0) Gecko/20100101 Firefox/102.0",

    # Safari (Mac)
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Version/16.2 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Version/15.1 Safari/537.36",

    # Edge (Windows)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.0.0",

    # Opera
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/97.0.4719.63",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 OPR/97.0.4719.63",
    
    # Android (Chrome)
    "Mozilla/5.0 (Linux; Android 12; SM-G998B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 11; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.101 Mobile Safari/537.36",
    
    # Android (Firefox)
    "Mozilla/5.0 (Android 12; Mobile; rv:109.0) Gecko/109.0 Firefox/109.0",
    "Mozilla/5.0 (Android 10; Mobile; rv:102.0) Gecko/102.0 Firefox/102.0",
    
    # iPhone (Safari)
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Mobile/15E148 Safari/604.1",
    
    # iPad (Safari)
    "Mozilla/5.0 (iPad; CPU OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 15_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Mobile/15E148 Safari/604.1",
]


# Chrome 옵션 설정
options = Options()
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")  # 랜덤 User-Agent 적용

# WebDriver 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

#channel_name_li=[]
#subscriber_count_li=[]
video_count_li=[]

error_count = 0

for i in tqdm(range(len(data))):
    try:
        # 크롤링하려는 YouTube 채널 URL
        channel_url = f'https://www.youtube.com/{data[i]}'

        # 채널 페이지 열기
        driver.get(channel_url)

        # 페이지 로딩 대기 (동적 콘텐츠가 로딩될 시간을 줍니다)
        time.sleep(1)

        # # 채널명 가져오기
        # try:
        #     channel_name = driver.find_element(By.XPATH, '//meta[@property="og:title"]').get_attribute('content')
        # except:
        #     channel_name = "N/A"
        # channel_name_li.append(channel_name)

        # # 구독자 수 가져오기
        # try:
        #     subscriber_count = driver.find_element(
        #         By.XPATH, '//*[@id="page-header"]/yt-page-header-renderer/yt-page-header-view-model/div/div[1]/div/yt-content-metadata-view-model/div[2]/span[1]'
        #     ).text
        # except:
        #     subscriber_count = "N/A"
        # subscriber_count_li.append(subscriber_count)

        # 총 동영상 수 가져오기
        try:
            video_count = driver.find_element(
                By.XPATH, '//*[@id="page-header"]/yt-page-header-renderer/yt-page-header-view-model/div/div[1]/div/yt-content-metadata-view-model/div[2]/span[3]').text
            #print(video_count)
            df.iloc[3501+i,11]=video_count
        except:
            continue

    except Exception as e:
        print(f"채널 크롤링 중 오류 발생: {e}")
        error_count += 1
        continue  # 오류 발생 시 다음 채널로 이동
        
        if error_count >= 3:  # 3번 연속 오류 발생하면 종료
            print("연속 3회 오류 발생. 크롤링을 중단합니다.")
            break

# 브라우저 종료
driver.quit()

# channel_id = data[:len(subscriber_count_li)]
# channel_sub = pd.DataFrame({"channel_id": channel_id, "subscriber": subscriber_count_li})
#df.to_csv('channel_subscriber.csv', index=False, encoding="utf-8-sig")

  3%|██▎                                                                          | 188/6180 [13:51<7:21:35,  4.42s/it]


KeyboardInterrupt: 

In [147]:
pd.set_option('display.max_rows', None)
df.iloc[3500:,8:]

,channel_id,subscriber_count,category,total_video_count
3500,"('Wooang 우앙', '@wooang')",622000.0,음식/요리,동영상 1.5천개
3501,"('상해기SangHyuk', '@sanghyuk1109')",1580000.0,음식/요리,동영상 1천개
3502,"('Eating Tv', '@eatingtv')",19800.0,음식/요리,동영상 1.1천개
3503,"('Eat with Boki', '@EatwithBoki')",10300000.0,음식/요리,동영상 999개
3504,"('군군이', '@goongoon.e')",69500.0,음식/요리,동영상 920개
3505,"('푸먹_foomuk', '@foomuk')",1510000.0,음식/요리,동영상 89개
3506,"('쏘영 Ssoyoung', '@Ssoyoung')",11700000.0,음식/요리,동영상 1.3천개
3507,"('Hongyu ASMR 홍유', '@HongyuASMR')",16500000.0,음식/요리,동영상 956개
3508,"('KNN NEWS', '@KNN_NEWS')",836000.0,음식/요리,동영상 10만개
3509,"(""Heungsam's Family"", '@heungsamsfamily')",3700000.0,음식/요리,동영상 2.3천개


In [149]:
df.to_csv('video_datas.csv', index=False, encoding="utf-8-sig")

In [15]:
import pandas as pd
df_sub=pd.DataFrame({'channel_id':data,
                     'subscriber_count':[s.replace('구독자', '').replace('명', '').strip() for s in subscriber_count_li],
                     'total_video_count':[s.replace('동영상', '').replace('개', '').strip() for s in video_count_li]})
df_sub

,channel_id,subscriber_count,total_video_count
0,"[닥터셰프, UCc4GDo3-a9FyGd8_zGpYJZQ]",128,38
1,"[주부나라, UCzIOIStANWRgYa3BRuJOg8Q]",161만,1.7천
2,"[이 남자의 cook, UCy2WX3w5UyYFHBDHyWFKNUQ]",249만,987
3,"[KBS Entertain: 깔깔티비, UC271vX3c3kYZsgpz7-zPBoA]",106만,8.2천
4,"[유라야놀자(Let's play YURA), UCx8IhwapX8E7uooFYJIe...",227만,2천
...,...,...,...
190,"[탑텐사커, UC0Ue4AZLKcqDl2hqZn4eP4g]",1.76만,261
191,"[스포츠 스토리, UCVl5dHKhpf4Z07HIH_KbnWQ]",4.54만,308
192,"[Snooker Fan Zone, UC9nUNwRwWBoPZHNtRzDUp8g]",7.62천,73
193,"[초이스포츠, UCbcl1BPu7sLvaznemwlwu1A]",737,49


In [17]:
df_pre=pd.read_csv('channel_info.csv',encoding='utf-8-sig')
df_total=pd.concat([df_pre,df_sub]).reset_index(drop=True)
df_total

,channel_id,subscriber_count,total_video_count
0,"['보겸TV', 'UCu9BCtGIEr73LXZsKmoujKw']",1560만,9.3천
1,"['JYP Entertainment', 'UCaO6TYtlC8U5ttz62hTrZgg']",3040만,2.3천
2,"['늘이농', 'UCoFfV8sTi5PWKEdvJP2XzWQ']",37.5만,451
3,"['planD플랜디', 'UCPaApoLkmPqxCeZF7gjoc3w']",225만,256
4,"['JTBC Drama', 'UCkbJc8jMcTXwhtmN5VMwfXg']",493만,3.2만
...,...,...,...
597,"[탑텐사커, UC0Ue4AZLKcqDl2hqZn4eP4g]",1.76만,261
598,"[스포츠 스토리, UCVl5dHKhpf4Z07HIH_KbnWQ]",4.54만,308
599,"[Snooker Fan Zone, UC9nUNwRwWBoPZHNtRzDUp8g]",7.62천,73
600,"[초이스포츠, UCbcl1BPu7sLvaznemwlwu1A]",737,49


In [19]:
df_total.to_csv('channel_info.csv', index=False, encoding='utf-8-sig')

### 채널정보 칼럼 추가

In [141]:
df['channel_id'] = df['channel_id'].astype(str)
df_total['channel_id'] = df_total['channel_id'].astype(str)
df=pd.merge(left=df, right=df_total, how='inner', on='channel_id')
df

,title,video_id,published_date,thumbnail_url,view_count,...,person_positions,prob_text,contain_object,subscriber_count,total_video_count
0,"ENG│코스트코 고기 2.5키로 소분하는 자취 일상. 대파꼬치전, 또띠아와퍼 만들고...",uxDZyUHO9sE,2024-08-18,https://i.ytimg.com/vi/uxDZyUHO9sE/hqdefault.jpg,1113043.0,...,[middle middle l],[해당 없음],"[bottle, bottle, bottle, bottle, bowl, person,...",225만,256
1,"＂이불을 한 채만 쓸까요?＂ 더 가까워진 임지연♥추영우, 찐-한 합방💞 | 옥씨부인...",n9TTmAATM1M,2025-01-19,https://i.ytimg.com/vi/n9TTmAATM1M/hqdefault.jpg,1975060.0,...,[사람 없음],[해당 없음],[],493만,3.2만
2,더 이상 물러날 곳❌ 의금부에 추포된 하율리-최정우 | 옥씨부인전 16회 | JTB...,6L66YeMkkLQ,2025-01-26,https://i.ytimg.com/vi/6L66YeMkkLQ/hqdefault.jpg,1248961.0,...,"[middle middle l, middle middle l]",[해당 없음],"[person, person]",493만,3.2만
3,[스페셜 선공개] ＂현감 얼굴에 칼자국..＂ 임지연 향한 윤지혜의 의미심장한 질문....,Xu6MTsCNunA,2025-01-15,https://i.ytimg.com/vi/Xu6MTsCNunA/hqdefault.jpg,1093319.0,...,[middle middle l],[해당 없음],[person],493만,3.2만
4,"그렇게 유난을 떨던 공개 장기연애 끝, 한달 뒤 떠난 도쿄여행에서 생긴 일💧",ehUnAK-uNqk,2025-01-12,https://i.ytimg.com/vi/ehUnAK-uNqk/hqdefault.jpg,341820.0,...,[middle middle l],[해당 없음],[person],13.5만,85
5,"VLOG • 연휴동안 생긴 일✨ 자취요리, smp 리터치, 어색한 썸남보이랑 데이트...",ACdNDBZw_4c,2025-02-17,https://i.ytimg.com/vi/ACdNDBZw_4c/hqdefault.jpg,88353.0,...,"[middle middle l, left middle m, middle middle m]","[new!, 연휴의]","[person, person, cup, person]",13.5만,85
6,"VLOG • 두피문신 smp 후기👩🏻‍🦲, 레전드 야장맛집 뿌시기, 행궁동과 홍대...",eZVo_G-S_5g,2024-10-12,https://i.ytimg.com/vi/eZVo_G-S_5g/hqdefault.jpg,84381.0,...,"[middle middle l, left middle l, middle up s]",[해당 없음],"[person, cup, cup, person, person, cup]",13.5만,85
7,"VLOG • 그래요 나 연애해요,, 예뻐야 하는 날 야무지게 준비하고🧖🏻‍♀️ 남친...",qiuSUSiyGkI,2025-03-10,https://i.ytimg.com/vi/qiuSUSiyGkI/hqdefault.jpg,77976.0,...,"[middle middle l, middle up m, left up s, left...",[브이로그;],"[person, fork, person, person, dining table, c...",13.5만,85
8,"VLOG • 2달 차 솔로 일상😇 기어코 이사, 대전여행, 청첩장모임, 집밥, 행궁...",WXgiwBUPhyc,2025-01-24,https://i.ytimg.com/vi/WXgiwBUPhyc/hqdefault.jpg,58725.0,...,[left middle s],[해당 없음],"[person, bed, cup]",13.5만,85
9,"입냄새의 진짜 원인, 목에 낀 똥을 싹 없애는 간단한 방법 (이영섭 원장 2부)",dR5scpolbQY,2024-05-10,https://i.ytimg.com/vi/dR5scpolbQY/hqdefault.jpg,4179526.0,...,[middle middle l],"[(이비인무과 영업비일), 입냄새]",[person],266만,979


In [ ]:
def convert_subscriber_count(value):
    if '만' in value:
        return float(value.replace('만', '')) * 10000
    return float(value)

df['subscriber_count'] = df['subscriber_count'].apply(convert_subscriber_count)
df['total_video_count'] = df['total_video_count'].apply(convert_subscriber_count)

In [75]:
df=pd.read_csv("video_datas.csv", encoding="utf-8-sig")
df

,title,video_id,published_date,thumbnail_url,view_count,like_count,comment_count,duration,channel_id,subscriber_count,category,total_video_count
0,결국 긴팔문신 제거 하는 조두팔,t-zwVkTOZA8,20250328,https://i.ytimg.com/vi/t-zwVkTOZA8/maxresdefau...,244420.0,NaN,221.0,7:39,"('조두팔', '@조두팔이라고')",412000.0,일상/브이로그,NaN
1,[vlog] 고등학교 교사 직장인 브이로그 | 학부모 총회 그리고 상담 가득한 한 ...,AMwSEsFVRGg,20250328,https://i.ytimg.com/vi/AMwSEsFVRGg/maxresdefau...,11267.0,265.0,54.0,18:37,"('HappyHojin', '@Happppyhojin')",46500.0,일상/브이로그,NaN
2,오사카&교토 브이로그 • 쇼핑에미띤여자들,PocqBywf0FU,20250224,https://i.ytimg.com/vi/PocqBywf0FU/maxresdefau...,12158.0,315.0,110.0,21:46,"('권예왕왕', '@ye_one_e')",15700.0,일상/브이로그,NaN
3,마라로제엽떡+닭꼬치 꿀조합•노티드 딸기전메뉴🍓먹고 결국 컵라면2개로 마무리•옛날통닭...,6eNhJ4LYr1k,20250329,https://i.ytimg.com/vi/6eNhJ4LYr1k/maxresdefau...,14333.0,441.0,71.0,29:32,"('째링', '@buttermellowday')",71700.0,일상/브이로그,NaN
4,[썰레디윗미] 이젠 말할 수 있다🙄 역대급 똥차 썰 낋여오니라.ㅣ이게 실화라고? 구...,G9tXGudpOCo,20250328,https://i.ytimg.com/vi/G9tXGudpOCo/maxresdefau...,58409.0,NaN,118.0,18:17,"('냔지 nyanji', '@quya_a')",296000.0,일상/브이로그,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9676,프랑스 오세르에서의 로맨틱한 호텔 꿀팁과 아름다운 순간들,m9ktsc4qXgw,20250321,https://i.ytimg.com/vi/m9ktsc4qXgw/maxresdefau...,12.0,0.0,1.0,4:48,"('인생호텔', '@lifehotel00')",36.0,여행,NaN
9677,세계에서 가장 작은 나라의 가장 작은 호텔,ubnBh-HRcrc,20231225,https://i.ytimg.com/vi/ubnBh-HRcrc/maxresdefau...,2795525.0,24993.0,1200.0,22:31,"('서재로36', '@서재로36')",550000.0,여행,NaN
9678,해즈빈 호텔 인물 탐구 | 만능 싸이코 사슴 -📻알래스터📻,WTDMvhnKo9I,20240706,https://i.ytimg.com/vi/WTDMvhnKo9I/maxresdefau...,174618.0,2642.0,143.0,21:15,"('미스터 폴보', '@MrFolvo')",4860.0,여행,NaN
9679,"키프로스 페이아에서의 꿈같은 호텔 휴식, 당신의 여행을 특별하게 만들어줄 장소",-tHxjBrV534,20250329,https://i.ytimg.com/vi/-tHxjBrV534/maxresdefau...,2.0,0.0,1.0,4:37,"('호텔탐험대', '@호텔탐험대')",14.0,여행,NaN
